In [0]:
dbutils.fs.ls("/Volumes/dev/demo_db/vol1")

In [0]:
base_dir = "/Volumes/dev/demo_db/vol1"

In [0]:
flight_Schema_ddl = """
            FL_DATE DATE,
            OP_CARRIER STRING,
            OP_CARRIER_FL_NUM INT,
            ORIGIN STRING,
            ORIGIN_CITY_NAME STRING,
            DEST STRING,
            DEST_CITY_NAME STRING,
            CRS_DEP_TIME INTEGER,
            DEP_TIME INTEGER,
            WHEELS_ON INTEGER,            
            TAXI_IN INTEGER,
            CRS_ARR_TIME INT,
            ARR_TIME INT,
            CANCELLED STRING,
            DISTANCE INT
            """

In [0]:
flight_time_df = (spark.read.format("json")
                  .schema(flight_Schema_ddl)
                  .option("dateFormat","M/d/y")
                  .load(f"{base_dir}/flight-time.json"))

In [0]:
flight_time_df.display()

In [0]:
%sql
create table if not exists dev.demo_db.flight_time_tbl(
            FL_DATE DATE,
            OP_CARRIER STRING,
            OP_CARRIER_FL_NUM INT,
            ORIGIN STRING,
            ORIGIN_CITY_NAME STRING,
            DEST STRING,
            DEST_CITY_NAME STRING,
            CRS_DEP_TIME INTEGER,
            DEP_TIME INTEGER,
            WHEELS_ON INTEGER,            
            TAXI_IN INTEGER,
            CRS_ARR_TIME INT,
            ARR_TIME INT,
            CANCELLED STRING,
            DISTANCE INT
) USING DELTA;

In [0]:
flight_time_df.write.format("delta").mode("append").saveAsTable("dev.demo_db.flight_time_tbl")

In [0]:
%sql
select * from dev.demo_db.flight_time_tbl

Delta Table Builder API Method

In [0]:
from delta import DeltaTable

(DeltaTable.createOrReplace(spark)
    .tableName("dev.demo_db.flight_time_tbl")
    .addColumns(flight_Schema_ddl)
    .execute()
 )

In [0]:
%sql
select * from dev.demo_db.invoices_yearly

External Location Tables

In [0]:
%sql
create external location if not exists ext_data
URL 'abfss://rvazdb-metastore@rvchandruazdbsa1.dfs.core.windows.net/ext_data'
with (credential `custom_connection`)

In [0]:
flight_time_df.coalesce(1).write.format("delta").mode("overwrite").save("abfss://rvazdb-metastore@rvchandruazdbsa1.dfs.core.windows.net/ext_data")

In [0]:
%fs ls abfss://rvazdb-metastore@rvchandruazdbsa1.dfs.core.windows.net/ext_data